In [33]:
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Notebook code based on learn.co Convolutional Neural Networks Code Along

# Pre-Processing

In [34]:
train_folder = 'data/seg_train/seg_train'
test_folder = 'data/seg_test/seg_test'
validation_folder = 'data/seg_pred/seg_pred'
im_classes = ['buildings','forest', 'glacier', 'mountain', 'sea', 'street']

In [35]:
train_imgs = []
test_imgs = []
validation_imgs = []

for im_class in im_classes:
    train_imgs.extend([file for file in os.listdir(train_folder+'/'+im_class) if file.endswith('.jpg')])
    test_imgs.extend([file for file in os.listdir(test_folder+'/'+im_class) if file.endswith('.jpg')])
    
validation_imgs = [file for file in os.listdir(validation_folder) if file.endswith('.jpg')]
    
train_batch_size = len(train_imgs)
test_batch_size = len(test_imgs)
validation_batch_size = len(validation_imgs)

In [36]:
print(train_batch_size)
print(test_batch_size)
print(validation_batch_size)

14034
3000
7301


In [71]:
# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator().flow_from_directory(
        test_folder, 
        target_size=(64, 64), batch_size = test_batch_size) 

# get all the data in the directory split/validation (200 images), and reshape them
#val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
#        validation_folder, 
#        target_size=(64, 64), batch_size = validation_batch_size)

# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_folder, 
        target_size=(64, 64), batch_size = train_batch_size)

Found 3000 images belonging to 6 classes.
Found 14034 images belonging to 6 classes.


In [73]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
#val_images, val_labels = next(val_generator)

In [74]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
#m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
#print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
#print ("val_images shape: " + str(val_images.shape))
#print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 14034
Number of testing samples: 3000
train_images shape: (14034, 64, 64, 3)
train_labels shape: (14034, 6)
test_images shape: (3000, 64, 64, 3)
test_labels shape: (3000, 6)


In [75]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
#val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
#print(val_img.shape)

(14034, 12288)
(3000, 12288)


In [76]:
train_y = np.reshape(train_labels[:,0], (train_batch_size,1))
test_y = np.reshape(test_labels[:,0], (test_batch_size,1))
#val_y = np.reshape(val_labels[:,0], (200,1))

# Baseline Neural Network

In [77]:
# Build a baseline fully connected model
from keras import models
from keras import layers
np.random.seed(123)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [81]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                    train_y,
                    epochs=5,
                    batch_size=32,
                    validation_data=(test_img, test_y)
                                    )

Epoch 1/5
439/439 [==============================] - 3s 6ms/step - loss: 0.4332 - accuracy: 0.8439 - val_loss: 0.4156 - val_accuracy: 0.8543
Epoch 2/5
439/439 [==============================] - 3s 6ms/step - loss: 0.4332 - accuracy: 0.8439 - val_loss: 0.4156 - val_accuracy: 0.8543
Epoch 3/5
439/439 [==============================] - 3s 6ms/step - loss: 0.4332 - accuracy: 0.8439 - val_loss: 0.4156 - val_accuracy: 0.8543
Epoch 4/5
439/439 [==============================] - 3s 7ms/step - loss: 0.4332 - accuracy: 0.8439 - val_loss: 0.4155 - val_accuracy: 0.8543
Epoch 5/5
439/439 [==============================] - 3s 7ms/step - loss: 0.4332 - accuracy: 0.8439 - val_loss: 0.4155 - val_accuracy: 0.8543


# Convolutional Neural Network

In [82]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [83]:
history = model.fit(train_images,
                    train_y,
                    epochs=10,
                    batch_size=32,
                    validation_data=(test_images, test_y))

Epoch 1/10
439/439 [==============================] - 119s 271ms/step - loss: nan - acc: 0.8380 - val_loss: nan - val_acc: 0.8543
Epoch 2/10
439/439 [==============================] - 121s 276ms/step - loss: nan - acc: 0.8439 - val_loss: nan - val_acc: 0.8543
Epoch 3/10
439/439 [==============================] - 126s 287ms/step - loss: nan - acc: 0.8439 - val_loss: nan - val_acc: 0.8543
Epoch 4/10
439/439 [==============================] - 132s 301ms/step - loss: nan - acc: 0.8439 - val_loss: nan - val_acc: 0.8543
Epoch 5/10
439/439 [==============================] - 132s 300ms/step - loss: nan - acc: 0.8439 - val_loss: nan - val_acc: 0.8543
Epoch 6/10
439/439 [==============================] - 136s 310ms/step - loss: nan - acc: 0.8439 - val_loss: nan - val_acc: 0.8543
Epoch 7/10
439/439 [==============================] - 136s 309ms/step - loss: nan - acc: 0.8439 - val_loss: nan - val_acc: 0.8543
Epoch 8/10
439/439 [==============================] - 234s 533ms/step - loss: nan - acc: 0